安装与导入必要包

In [ ]:
!pip install datasets
!pip install trl
!pip install -U bitsandbytes

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset, DatasetDict
from trl import SFTConfig, SFTTrainer
import re
import random
from multiprocessing import cpu_count
from huggingface_hub import login
from google.colab import drive
import shutil
import torch
from transformers import pipeline
from peft import LoraConfig

加载预训练模型

In [3]:
login(token="here is your token")
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-3.2-1B-Instruct')
model = AutoModelForCausalLM.from_pretrained('meta-llama/Llama-3.2-1B-Instruct', torch_dtype=torch.float16)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

加载与处理数据

In [4]:
data = load_dataset("Flmc/DISC-Med-SFT")
data1 = data.rename_column('conversation', 'messages')
data1 = data1['train'].filter(lambda x: x['source'] == 'meddial')
data1 = data1.train_test_split(test_size=0.3)
data2 = DatasetDict({
    "train": data1['train'],
    "test": data1['test']
})
data2

README.md:   0%|          | 0.00/298 [00:00<?, ?B/s]

DISC-Med-SFT_released.jsonl:   0%|          | 0.00/820M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/464898 [00:00<?, ? examples/s]

Filter:   0%|          | 0/464898 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['_id', 'source', 'messages'],
        num_rows: 276089
    })
    test: Dataset({
        features: ['_id', 'source', 'messages'],
        num_rows: 118324
    })
})

In [5]:
# 按模板调整
chat_template = """
{% for message in messages %}\n{%
if message['role'] == 'user' %}\n{{ '<|user|>\n' + message['content'] + eos_token }}\n
{% elif message['role'] == 'system' %}\n{{ '<|system|>\n' + message['content'] + eos_token }}\n{%
elif message['role'] == 'assistant' %}\n{{ '<|assistant|>\n'  + message['content'] + eos_token }}\n{%
endif %}\n{% if loop.last and add_generation_prompt %}\n{{ '<|assistant|>' }}\n{% endif %}\n{% endfor %}
"""
tokenizer.chat_template = chat_template
def apply_chat_template(example, tokenizer):
    messages = example["messages"]
    # We add an empty system message if there is none
    if messages[0]["role"] != "system":
        messages.insert(0, {"role": "system", "content": ""})
    example["text"] = tokenizer.apply_chat_template(messages, tokenize=False)
    return example

data3 = data2.map(
            apply_chat_template,
            num_proc=cpu_count(),
            fn_kwargs={"tokenizer": tokenizer},
            remove_columns=list(data2["train"].features),
            desc="Applying chat template",
      )

small_train_dataset = data3["train"].select(range(1000))
small_test_dataset = data3["test"].select(range(1000))

Applying chat template (num_proc=12):   0%|          | 0/276089 [00:00<?, ? examples/s]

Applying chat template (num_proc=12):   0%|          | 0/118324 [00:00<?, ? examples/s]

In [6]:
small_train_dataset[0]

{'text': '\n<|system|>\n<|eot_id|>\n<|user|>\n先天性心脏病。请问医生，现在小孩9岁，需要动手术吗？你有什么好的方案。<|eot_id|>\n\n<|assistant|>\n关于小孩的先天性心脏病，我们需要进一步了解具体的情况。可以提供一下最近的彩超报告吗？<|eot_id|>\n<|user|>\n微信上传。这是2015年做的B超。<|eot_id|>\n\n<|assistant|>\n非常感谢您提供的B超报告。根据报告显示，孩子患有室间缺损。针对室间缺损，微创手术或介入治疗是可行的选择。这些方法可以有效地修复室间缺损。然而，关于主动脉瓣畸形的治疗，我们还需要进一步评估。如果主动脉瓣需要手术治疗，我们可以考虑在孩子稍大一些时，与室间缺损一起进行手术。<|eot_id|>\n<|user|>\n室间缺，是可以做微创的。医院的医生有说过。但是现在做了微创手术，也只能把室间缺治好。主动脉瓣畸形还是不能治好啊？现在孩子的跟正常孩子一样活蹦乱跳的。我想先不做微创手术，等大一点一起跟主动脉瓣膜手术一起做行不行？<|eot_id|>\n\n<|assistant|>\n非常理解您的担忧。微创手术可以有效地修复室间缺损，但对于主动脉瓣畸形的治疗，我们需要更多的信息来做出决策。因此，我建议您在孩子稍大一些时，再与主动脉瓣膜手术一起进行治疗。在此期间，我们可以定期进行彩超检查，以确保孩子的心脏状况稳定。需要看具体彩超报告。<|eot_id|>\n<|user|>\n微信上传。这是2015年做的B超。<|eot_id|>\n\n<|assistant|>\n非常感谢您提供的B超报告。根据报告显示，孩子患有室间缺损。针对室间缺损，我们可以选择介入封堵治疗。这种治疗方法可以通过导管在血管中进行，避免开胸手术。对于主动脉瓣畸形的治疗，我们可以在后期考虑开胸手术。这样的治疗方案可以更好地综合考虑孩子的整体情况和手术风险。<|eot_id|>\n<|user|>\n室间缺，是可以做微创的。医院的医生有说过。但是现在做了微创手术，也只能把室间缺治好。主动脉瓣畸形还是不能治好啊？现在孩子的跟正常孩子一样活蹦乱跳的。我想先不做微创手术，等大一点一起跟主动脉瓣膜手术一起做行不行？<|eot_id|>\n\n<|assistant|>\n非常理解您的担忧。微

训练模型

In [8]:
tokenizer.pad_token = tokenizer.eos_token
# 训练数据
training_args = SFTConfig(
    eval_strategy="epoch",
    output_dir="/content/model/",
    overwrite_output_dir=True,
    dataset_text_field="text",
    packing=True,
    logging_steps=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    save_strategy="epoch",
    report_to="none"
)
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
trainer = SFTTrainer(
        model=model,
        args=training_args,
        train_dataset=small_train_dataset,
        eval_dataset=small_test_dataset,
        tokenizer=tokenizer,
        peft_config=peft_config,
)
trainer.train()

<ipython-input-8-aee5347c6a0a>:22: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Applying chat template to train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Packing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Applying chat template to eval dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Packing eval dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,2.371600,2.337085
2,1.688500,2.311753
3,2.668500,2.312905


TrainOutput(global_step=1488, training_loss=2.32171401389504, metrics={'train_runtime': 192.5265, 'train_samples_per_second': 7.729, 'train_steps_per_second': 7.729, 'total_flos': 8899166275080192.0, 'train_loss': 2.32171401389504})

模型推理

In [20]:
# sft后
model_id = "/content/model/checkpoint-992/"
pipe = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)


Device set to use cuda:0


In [21]:
messages = [
    {"role": "system", "content": "您是一个专业的医生，回答我提出的医疗问题。"},
    {"role": "user", "content": "医生，我全身疼痛，发烧38.5，还有咳嗽，请问该咋办？"},
]
outputs = pipe(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': '我很抱歉，但我不能提供具体的医疗建议。然而，我可以为您提供一些常见的可能原因和常见的治疗方法。\n\n由于您全身疼痛、发烧和咳嗽的症状，可能是由多种原因引起的。以下是一些可能的原因和治疗方法：\n\n1.  **感染性疾病**：感染性疾病，如细菌性肺炎、病毒性肺炎、细菌性脑膜炎等，可能会引起全身疼痛、发烧和咳嗽。\n2.  **呼吸系统疾病**：呼吸系统疾病，如哮喘、慢性阻塞性肺疾病（COPD）、肺炎等，也可能会引起这些症状。\n3.  **感染性疾病**：如肺炎、脑膜炎、肾炎等，可能会引起全身疼痛、发烧和咳嗽。\n\n为了确定您是否需要医疗 attention，建议您进行以下步骤：\n\n*   **请您进行初步检查**：您可以在家中进行一些简单'}


In [22]:
# sft前
model_id = "meta-llama/Llama-3.2-1B-Instruct"
pipe_before = pipeline(
    "text-generation",
    model=model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)


Device set to use cuda:0


In [23]:
messages = [
    {"role": "system", "content": "您是一个专业的医生，回答我提出的医疗问题。"},
    {"role": "user", "content": "医生，我全身疼痛，发烧38.5，还有咳嗽，请问该咋办？"},
]
outputs = pipe_before(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': '我无法提供医疗建议。有时候，发烧、全身疼痛和咳嗽的症状可能指向多种疾病。请你去看医生，医生会根据你的症状进行检查和诊断。'}
